In [1]:
%matplotlib inline
seed = 1234
import pandas as pd
import os, glob
import numpy as np
np.random.seed(seed)
import random
random.seed(seed)
# fix random seed for reproducibility
import tensorflow as tf
tf.compat.v1.random.set_random_seed(seed)
import keras
from numpy import array
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import text_utilities as tu
import modeling_utils as mu
import image_utilities as iu
import ocr
import doc_classifier_model as dcm
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras import backend as K
import json

sess_config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1,
allow_soft_placement=True, device_count = {'CPU': 1})
sess = tf.Session(graph=tf.get_default_graph(),config=sess_config)
K.set_session(sess)

plt.style.use('ggplot')


Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.15.0
2.3.0


## **OCR function applied on images**

In [3]:
#data_path = '/home/sureclaim/Documents/Claims/real_data/'
#data_df = ocr.create_data_df(data_path)
#data_df.to_csv('/home/sureclaim/Documents/Claims/data_df.csv')

In [4]:
data_df = pd.read_csv('data/data_df_new.csv')
#data_df = pd.read_csv('/home/sureclaim/Documents/Claims/data_df.csv')
data_df.shape

(1946, 5)

In [5]:
print("NA counts:", data_df.isna().sum())
data_df = data_df.dropna()

NA counts: Unnamed: 0     0
filename       0
path           0
x_txt         65
y              0
dtype: int64


# DV distribution

In [6]:
# One hot encode category
data_df['y_oneh'] = mu.onehot_encode(data_df.y)
mu.get_dv_dist(data_df, 'filename', 'y')

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


y,Aadhar Card,Diagnostic Bill,Discharge Summary,Insurance Card,Internal Case Papers,Pan Card,Phramacy Bill,Policy Copy,Prescriptions,Receipts
counts,112.00,114.00,268.00,111.0,277.00,46.00,306.00,145.00,312.00,190.0
perc (%),5.95,6.06,14.25,5.9,14.73,2.45,16.27,7.71,16.59,10.1


# Make Vocab

In [7]:
# Clean the text column; keep only alphabets
data_df['x_txt_cleaned'] = data_df.x_txt.apply(tu.clean_string)

# Setting max padded sequence length = 70
max_len = 70
vocab = tu.make_vocab(data_df.x_txt_cleaned)
w2i = tu.make_w2i(vocab)

Vocab size: 29366


# Train - Test Split

In [8]:
# Shuffle dataframe
data_df = data_df.sample(frac=1)

# Get word level splits
x_train, x_test, y_train, y_test = train_test_split(data_df[['path', 'x_txt_cleaned']], data_df.y_oneh, test_size=0.3, random_state=201)

print("----------------------------------------")
print("Train X Size:", x_train.shape)
print("Train Y Size:", y_train.shape)
print("----------------------------------------")
print("Test X size:", x_test.shape)
print("Test Y Size:", y_test.shape)

# TODO show DV dist for every sample

----------------------------------------
Train X Size: (1316, 2)
Train Y Size: (1316,)
----------------------------------------
Test X size: (565, 2)
Test Y Size: (565,)


# Make Tensors

In [9]:
# make tensors
x_txt_train = tu.make_tensor_np(x_train.x_txt_cleaned, w2i, max_len)
#x_img_train = np.stack(x_train.path.apply(rfe.generate_image_features, args=[resnet50]))
#np.save('data/img_features_train_new.npy', x_img_train)
x_img_train = np.load('data/img_features_train.npy')
y_train = np.stack(y_train.to_numpy())

x_txt_test = tu.make_tensor_np(x_test.x_txt_cleaned, w2i, max_len)
#x_img_test = np.stack(x_test.path.apply(rfe.generate_image_features, args=[resnet50]))
#np.save('data/img_features_test_new.npy', x_img_test)
x_img_test = np.load('data/img_features_test.npy')
y_test = np.stack(y_test.to_numpy())

print("X text train tensor shape:", x_txt_train.shape)
print("X images train tensor shape:", x_img_train.shape)
print("y train shape:", y_train.shape)
print("----------------------------------------")
print("X text test tensor shape:", x_txt_test.shape)
print("X images test tensor shape:", x_img_test.shape)
print("y test shape:", y_test.shape)

X text train tensor shape: (1316, 70)
X images train tensor shape: (1316, 131072)
y train shape: (1316, 10)
----------------------------------------
X text test tensor shape: (565, 70)
X images test tensor shape: (565, 131072)
y test shape: (565, 10)


# DV Distribution by Dataset

In [10]:
# TODO: make reports, write to csv
print("Train DV distribution:----------------------------------------------------------------------- ")
print(mu.get_dv_dist(y_train))
print("Test DV distribution:------------------------------------------------------------------------ ")
print(mu.get_dv_dist(y_test))

Train DV distribution:----------------------------------------------------------------------- 
            0     1      2     3      4     5      6      7      8      9
counts   85.0  77.0  191.0  78.0  199.0  33.0  213.0  107.0  205.0  128.0
perc(%)   6.5   5.9   14.5   5.9   15.1   2.5   16.2    8.1   15.6    9.7
Test DV distribution:------------------------------------------------------------------------ 
            0     1     2     3     4     5     6     7      8     9
counts   27.0  37.0  77.0  33.0  78.0  13.0  93.0  38.0  107.0  62.0
perc(%)   4.8   6.5  13.6   5.8  13.8   2.3  16.5   6.7   18.9  11.0


# Define callbacks

In [11]:

n_classes = y_test.shape[1]
vocab_size = len(vocab)

config = dict({'input_text_shape': max_len, 'input_img_shape': x_img_train.shape[1], 'n_classes': n_classes, 'vocab_size': vocab_size, 'w2i':w2i})

with open('model_config', 'w') as configfile:
    json.dump(config, configfile, indent=2)

doc_classifier = dcm.build_doc_classifier(max_len, x_img_train.shape[1], n_classes, vocab_size)

## Defining keras callbacks
log_dir = 'models/tf-log/'
tb_cb = TensorBoard(log_dir=log_dir, histogram_freq=0)

model_checkpoint = ModelCheckpoint('models/doc_classifier.ckpt', monitor='f1', save_weights_only=True, mode = 'max', save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor='f1', mode = 'max', patience=30, verbose=2)
reduce_lr = ReduceLROnPlateau(monitor='f1', mode = 'max', factor=0.5, patience=5, min_lr=0.00001, verbose=2)
cbks = [tb_cb, early_stopping, reduce_lr, model_checkpoint]


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Train

In [12]:
# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model
    # train LSTM
    history = doc_classifier.fit([x_img_train, x_txt_train], y_train,
                    batch_size=256, 
                    epochs=30, 
                    shuffle=False,
                    validation_split=0.1,
                    callbacks=cbks,
                    verbose=1)
    save_path = saver.save(sess, "models/sess2.ckpt")
    print("Sess saved in path: %s" % save_path)
    
    ##Save model weights
    doc_classifier.save_weights('models/doc_classifier2.h5')
    
    model_json = doc_classifier.to_json()
    with open('model2.json', 'w') as json_file:
        json_file.write(model_json)
    doc_classifier.save_weights('model2.h5')
    
    loss, accuracy, f1 = doc_classifier.evaluate([x_img_train, x_txt_train], y_train, verbose=2)
    print("Training Accuracy: ", round(accuracy*100, 2),"%" )
    print("Training F1: ", round(f1*100, 2),"%" )
    print("Training Loss: ", loss )
    print(mu.get_model_metrics(doc_classifier, [x_img_train, x_txt_train], y_train))
    print('-----------------------------------------------------------------------------------')
    loss, accuracy, f1 = doc_classifier.evaluate([x_img_test, x_txt_test], y_test, verbose=2)
    print("Test Accuracy: ", round(accuracy*100, 2),"%" )
    print("Test F1: ", round(f1*100, 2),"%" )
    print("Test Loss: ", loss )
    print(mu.get_model_metrics(doc_classifier, [x_img_test, x_txt_test], y_test))

#mu.plot_history(history)


Train on 1184 samples, validate on 132 samples


Epoch 1/30
1184/1184 [==============================] - 44s 38ms/step - loss: 8.0693 - accuracy: 0.1757 - f1: 0.1672 - val_loss: 4.9495 - val_accuracy: 0.3864 - val_f1: 0.1761


Epoch 00001: f1 improved from -inf to 0.16718, saving model to models/doc_classifier.ckpt
Epoch 2/30
1184/1184 [==============================] - 42s 36ms/step - loss: 5.2887 - accuracy: 0.3167 - f1: 0.2876 - val_loss: 4.1833 - val_accuracy: 0.4091 - val_f1: 0.3723

Epoch 00002: f1 improved from 0.16718 to 0.28756, saving model to models/doc_classifier.ckpt
Epoch 3/30
1184/1184 [==============================] - 44s 37ms/step - loss: 4.5067 - accuracy: 0.4079 - f1: 0.3841 - val_loss: 3.6497 - val_accuracy: 0.4318 - val_f1: 0.3646

Epoch 00003: f1 improved from 0.28756 to 0.38411, saving model to models/doc_classifier.ckpt
Epoch 4/30
1184/1184 [==============================] - 44s 37ms/step - loss: 3.8999 - accuracy: 0.4603 - f1: 0.4544 - val_loss: 3.2902 - val_

1184/1184 [==============================] - 44s 37ms/step - loss: 0.0764 - accuracy: 0.9856 - f1: 0.9830 - val_loss: 1.0813 - val_accuracy: 0.6742 - val_f1: 0.6888

Epoch 00028: f1 did not improve from 0.98657
Epoch 29/30
1184/1184 [==============================] - 44s 38ms/step - loss: 0.0688 - accuracy: 0.9882 - f1: 0.9855 - val_loss: 1.0617 - val_accuracy: 0.6667 - val_f1: 0.6862

Epoch 00029: f1 did not improve from 0.98657
Epoch 30/30
1184/1184 [==============================] - 45s 38ms/step - loss: 0.0632 - accuracy: 0.9907 - f1: 0.9905 - val_loss: 1.0701 - val_accuracy: 0.6818 - val_f1: 0.6582

Epoch 00030: f1 improved from 0.98657 to 0.99050, saving model to models/doc_classifier.ckpt
Sess saved in path: models/sess2.ckpt
Training Accuracy:  92.17 %
Training F1:  91.25 %
Training Loss:  0.27510066869411065
1316/1316 [==============================] - 8s 6ms/step
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        85
     

In [20]:
from keras_self_attention import SeqSelfAttention
from keras.optimizers import Adagrad

tf.reset_default_graph()
# Add ops to save and restore all the variables.
#saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json, custom_objects={'SeqSelfAttention': SeqSelfAttention})
    loaded_model.load_weights('model.h5')

    opt = Adagrad(lr = 1e-3)
        #sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=["accuracy"])

    graph = tf.get_default_graph()
    
    saver = tf.train.Saver()
    # Restore variables from disk.
    saver.restore(sess, "models/sess.ckpt")
    print("Session restored.")
    
    # Re-evaluate the model
    with graph.as_default():
        loss,acc = loaded_model.evaluate([x_img_train, x_txt_train], y_train, verbose=2)
    print("Restored model, train accuracy: {:5.2f}%".format(100*acc))
    with graph.as_default():
        loss,acc = loaded_model.evaluate([x_img_test, x_txt_test], y_test, verbose=2)
    print("Restored model, test accuracy: {:5.2f}%".format(100*acc))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/sess.ckpt
Session restored.
Restored model, train accuracy: 88.83%
Restored model, test accuracy: 37.52%


## Get model evaluation metrics


In [15]:
loss, accuracy, f1 = doc_classifier.evaluate([x_img_train, x_txt_train], y_train, verbose=2)
print("Training Accuracy: ", round(accuracy*100, 2),"%" )
print("Training F1: ", round(f1*100, 2),"%" )
print("Training Loss: ", loss )
print(mu.get_model_metrics(doc_classifier, [x_img_train, x_txt_train], y_train))
print('-----------------------------------------------------------------------------------')
loss, accuracy, f1 = doc_classifier.evaluate([x_img_test, x_txt_test], y_test, verbose=2)
print("Test Accuracy: ", round(accuracy*100, 2),"%" )
print("Test F1: ", round(f1*100, 2),"%" )
print("Test Loss: ", loss )
print(mu.get_model_metrics(doc_classifier, [x_img_test, x_txt_test], y_test))

FailedPreconditionError: Attempting to use uninitialized value bidirectional_1/forward_lstm_1/kernel
	 [[node bidirectional_1/forward_lstm_1/kernel/read (defined at /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:376) ]]

Caused by op 'bidirectional_1/forward_lstm_1/kernel/read', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()

In [21]:
model_json = doc_classifier.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
doc_classifier.save_weights('model.h5')

In [28]:
from keras_self_attention import SeqSelfAttention

#del loaded_model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json, custom_objects={'SeqSelfAttention': SeqSelfAttention})
loaded_model.load_weights('model.h5')

opt = Adam(lr = 1e-3)
#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=["accuracy"])

graph = tf.get_default_graph()
# Re-evaluate the model
loss,acc = loaded_model.evaluate([x_img_train, x_txt_train], y_train, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 52.96%


In [18]:
# Create a basic model instance
with open('model_config', 'r') as configfile:
    config = json.load(configfile)
    
model = dcm.build_doc_classifier(config['input_text_shape'], config['input_img_shape'], config['n_classes'], config['vocab_size'])

# Evaluate the model
loss, acc, f1 = model.evaluate([x_img_train, x_txt_train], y_train, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


# Loads the weights

model.load_weights("models/doc_classifier.ckpt")

# Re-evaluate the model
loss,acc, f1 = model.evaluate([x_img_train, x_txt_train], y_train, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Untrained model, accuracy: 10.03%
Restored model, accuracy: 81.84%
